<a href="https://colab.research.google.com/github/sdgroeve/Machine-Learning-Course-2days/blob/main/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# MNIST digit classification

## 1. The dataset

The MNIST dataset<sup>1</sup> (Modified National Institute of Standards and Technology dataset) is a large dataset containing pre-processed **28x28 pixel** images of handwritten digits. The dataset is widely used for teaching machine learning concepts.

<sub>[1] THE MNIST DATABASE of handwritten digits, Yann LeCun, Courant Institute, NYU Corinna Cortes, Google Labs, New York Christopher J.C. Burges, Microsoft Research, Redmond.</sub>

Let's load this dataset straight from the [openml.org](https://www.openml.org/) public repository with the following Scikit-learn function:

In [ ]:
from sklearn.datasets import fetch_openml

mnist = fetch_openml('mnist_784', as_frame=False, cache=True)

The `fetch_openml()` function returns a Python [dictionary-like object](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_openml.html).

The actual data can be obtained with the following keys:

- data: np.array, scipy.sparse.csr_matrix of floats, or pandas DataFrame
- target: np.array, pandas Series or DataFrame

This dataset contains 70000 labeled images of digits:

In [ ]:
print("#datapoints: {}".format(len(mnist.data)))

Let's explore the feature vectors:

In [ ]:
print("Length of feature vector: {}\n".format(len(mnist.data[0])))
print("Example of feature vector:\n")
print(mnist.data[0])

Each image in the dataset is represented by a 784 dimensional vector with one gray-scale value (0 means black, 255 means white) for each of the 28x28 pixels.

We can reshape this feature vector to the gray-scaled image as follows:

In [ ]:
mnist.data[0].reshape((28,28))

We will denote the feature vectors as `X` and the corresponding labels as `y`:

In [ ]:
X = mnist.data
y = mnist.target

print(X.dtype)
print(y.dtype)

Notice that the labels are Python objects (strings):

In [ ]:
print(y)

We convert these to numbers (integers):

In [ ]:
y = y.astype('int64')

We can use the Python [matplotlib](https://matplotlib.org/) library to plot the digit images in `X` (the label for each image is shown in the title):

In [ ]:
import matplotlib.pyplot as plt
from random import randint

# Display 9 randomly selected images
for c in range(1, 10):
    plt.subplot(3, 3,c)
    i = randint(0,X.shape[0])
    im = X[i].reshape((28,28))
    plt.axis("off")
    plt.title("Label = {}".format(y[i]))
    plt.imshow(im, cmap='gray')

It is 'best practice' in Machine Learning to normalize the feature values such that all features have values with the same 'not to large' scale.

Scikit-learn has functions to normalize features in `sklearn.preprocessing`. The most common ones are [`MinMaxScaler()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) and [`StandardScaler()`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html).

We will use the `MinMaxScaler()`:

In [ ]:
from sklearn.preprocessing import MinMaxScaler

X = MinMaxScaler().fit_transform(X)

print(X[0])

To be able to **properly** evaluate our trained model we need to first create an **independent test set** with images that are **not used** during model building:

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.50, shuffle=True, random_state=42)

## 2. The model

Now we are ready to define our model.

We will fit a [decision tree](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) model with a maximum tree depth equal to 5 and with at least 10 datapoints in each leaf:

In [ ]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_depth=5, min_samples_leaf=10)

Each Scikit-learn model has a function `fit()` that optimizes the modelparameters to minimize the cost function:

In [ ]:
clf.fit(X_train,y_train)

Each Scikit-learn model also has a function `predict()` that applies the fitted model to compute class lables for feature vectors:

In [ ]:
# Perform the predictions
y_predicted = clf.predict(X_test)

print(y_predicted)

## 3. Evaluation

Scikit-learn offers many [metrics](https://scikit-learn.org/stable/modules/model_evaluation.html) for evaluating the prediction performance. The most common metric is `accuracy`:

In [ ]:
from sklearn.metrics import accuracy_score

print("Accuracy = {}%".format(accuracy_score(y_test, y_predicted)*100))

To get more insight into the prediction errors for each class we can compute a confusion matrix:

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

ConfusionMatrixDisplay.from_predictions(y_test, y_predicted)
plt.show()

Now, let's take a look a the misclassified images in the test set:

In [ ]:
index = 0
misclassified_images = []
for label, predict in zip(y_test, y_predicted):
    if label != predict:
        misclassified_images.append(index)
    index +=1

print("Number of misclassified test set images: {}".format(len(misclassified_images)))

Let's plot some of these:

In [ ]:
plt.figure(figsize=(10,10))
for plot_index, bad_index in enumerate(misclassified_images[0:20]):
    p = plt.subplot(4,5, plot_index+1) # 4x5 plot

    p.imshow(X_test[bad_index].reshape(28,28), cmap=plt.cm.gray,
            interpolation='bilinear')
    p.set_xticks(()); p.set_yticks(()) # remove ticks

    p.set_title(f'Pred: {y_predicted[bad_index]}, Actual: {y_test[bad_index]}');

## 4. Hyperparameter optimization

Scikit-learn offers [many functions](https://scikit-learn.org/stable/modules/grid_search.html) for hyperparameter optimization. We will use `GridSearchCV()` that evaluates different hyperparamter value combinations using cross-validation.

In `GridSearchCV()` you define the hyperparameter values to consider in a Python dictionary:  

In [ ]:
from sklearn.model_selection import GridSearchCV

tune_grid = {
            "max_depth": [3,7,10],
            "min_samples_split": [100,1000]
        }

print(tune_grid)

Next, we can initialize `GridSearchCV()` just like any other model in Scikit-learn:

In [ ]:
clf_cv = GridSearchCV(clf, tune_grid, cv=5, verbose=3)

`GridSearchCV()` also has a  `fit()` function that fit all the models and evaluate their prediction performance with the cross-validation procedure:

In [ ]:
clf_cv.fit(X_train,y_train)

The fitted `GridSearchCV()` has attribute `best_estimator_` that contains the model with the highest cross-validation accuracy **retrained on all the data**.

It also has attribute `best_score_` that contains the cross-validation score (accuracy in this case) of the best estimator:

In [ ]:
print(clf_cv.best_estimator_)
print(clf_cv.best_score_)

A fitted `GridSearchCV()` also has the function `predict()` that applies `best_estimator_` to predict the classes:

In [ ]:
y_predicted = clf_cv.predict(X_test)

ConfusionMatrixDisplay.from_predictions(y_test, y_predicted)
plt.show()

Let's plot some of the misclassified images:

In [ ]:
index = 0
misclassified_images = []
for label, predict in zip(y_test, y_predicted):
    if label != predict:
        misclassified_images.append(index)
    index +=1

print("Number of misclassified test set images: {}".format(len(misclassified_images)))

plt.figure(figsize=(10,10))
for plot_index, bad_index in enumerate(misclassified_images[0:20]):
    p = plt.subplot(4,5, plot_index+1) # 4x5 plot

    p.imshow(X_test[bad_index].reshape(28,28), cmap=plt.cm.gray,
            interpolation='bilinear')
    p.set_xticks(()); p.set_yticks(()) # remove ticks

    p.set_title(f'Pred: {y_predicted[bad_index]}, Actual: {y_test[bad_index]}');